# Design Pattern

## Simple Factory
通過專門定義一個類（工廠）來創建其他類的實例，被創建的實例通常都有共同的父類

- 練習：做個簡易計算機，輸入兩個數字和運算符號，得到結果。
- 想法：為了方便取用且降低耦合性以防止牽一髮動全身，可讓介面取用「運算子」，做一個運算的superclass，而使用多態性，讓介面取用時實例化出合適的運算子，返回結果。

In [7]:
# 定義一個抽象父類別
class Operation:

    def __init__(self, numberA, numberB):
        self.numberA = numberA
        self.numberB = numberB

    # 定義出一個抽象方法
    def getResult(self):
        result = 0
        return result


# 定義各個可以實例化的生產線子類別
# 加法
class OperationAdd(Operation):
    def getResult(self):
        result = super().numberA + super().numberB
        return result

# 減法
class OperationSub(Operation):
    def getResult(self):
        result = super().numberA - super().numberB
        return result

# 乘法
class OperationMul(Operation):
    def getResult(self):
        result = super().numberA * super().numberB
        return result

# 除法
class OperationDiv(Operation):
    def getResult(self):
        if super().numberB == 0:
            print('除數不得為0')
        result = super().numberA / super().numberB
        return result


# 定義一個簡單運算工廠類別 - 做一個工廠，以後可以新增及移除生產線，也可以選擇要用那個生產線製造東西
class OperationFactory:
    def createOperate(self):
        if self == '+':
            return OperationAdd()
        elif self == '-':
            return OperationSub()
        elif self == '*':
            return OperationMul()
        elif self == '/':
            return OperationDiv()


In [9]:
oper = Operation(1,2)
oper
oper = OperationFactory.createOperate('+')
result = oper.getResult()

TypeError: __init__() missing 2 required positional arguments: 'numberA' and 'numberB'